In [1]:
data_var = '2024-01-06'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11711,2024-01-06,Taiwan P. League+,03:30,Formosa Dreamers,New Taipei Kings,1.67,2.12,188.5,1.83,1.83,-3.5,2.02,1.71,QZZ7PxWH,0.598802,0.471698,0.546448,0.546448,0.070501,159.026,59.411833,0.373598,3.0,0.000000,0.000000,134.07,1.564,0.387724,0.247905,69.0,144.024,24.415599,0.169525,0.6,1.341641,2.236068,150.72,1.438,0.171377,0.119177,-36.0,109,96,1.23,1.57,0.000,0.000,0.167915,0.000000,0.117535,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11712,2024-01-06,Turquia Superliga,12:00,Pinar Karsiyaka,Manisa,1.20,4.29,169.5,1.88,1.86,-11.5,2.03,1.68,IoIS0QWl,0.833333,0.233100,0.531915,0.537634,0.066434,131.670,25.768461,0.195705,2.4,1.341641,0.559017,124.08,1.412,0.240146,0.170075,46.0,131.326,26.280789,0.200119,1.8,1.643168,0.912871,91.00,1.600,0.173638,0.108523,-22.0,94,65,1.32,1.40,0.000,0.000,0.795978,0.007563,0.133416,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11713,2024-01-06,Turquia Superliga,09:30,Galatasaray,Tofas,1.58,2.32,164.5,1.85,1.89,-4.5,1.99,1.72,xz1V16Hr,0.632911,0.431034,0.540541,0.529101,0.063946,140.144,53.349271,0.380675,1.2,1.643168,1.369306,95.04,1.744,0.639085,0.366448,-30.0,159.112,51.250802,0.322105,1.8,1.643168,0.912871,139.04,1.914,0.910456,0.475682,-1.0,72,88,1.32,1.58,0.000,0.000,0.268338,0.015125,0.102921,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11714,2024-01-06,Turquia Superliga,07:00,Samsunspor,Besiktas,3.46,1.29,151.5,1.83,1.91,6.5,2.01,1.68,zRE36n9R,0.289017,0.775194,0.546448,0.523560,0.064211,244.582,85.853439,0.351021,0.6,1.341641,2.236068,228.90,3.294,1.079829,0.327817,-25.0,145.828,34.424940,0.236065,2.4,1.341641,0.559017,101.64,1.830,0.620927,0.339304,18.0,70,77,3.27,1.32,0.000,0.000,0.646072,0.030251,0.126474,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
11715,2024-01-06,Taiwan Liga T1,05:30,Kaohsiung Aquas,Taoyuan Leopards,1.99,1.74,206.5,1.85,1.85,1.5,1.85,1.85,lftYPEok,0.502513,0.574713,0.540541,0.540541,0.077225,207.530,48.425046,0.233340,2.4,1.341641,0.559017,162.41,1.966,0.518729,0.263850,54.0,172.964,44.293878,0.256087,1.8,1.643168,0.912871,164.40,1.612,0.405117,0.251313,-8.0,109,120,1.49,1.37,0.000,172.964,0.094786,0.000000,0.000000,0.00,0.000,inf,0.0,0.0,0.0,0.26,0.052,14.230769,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,2024-01-06,Eua Ncaa,22:00,Arizona State,Colorado,2.35,1.59,146.5,1.83,1.83,4.5,1.73,0.00,judXg8nS,0.425532,0.628931,0.546448,0.546448,0.054463,191.848,53.346583,0.278067,1.8,1.643168,0.912871,245.18,2.898,0.694024,0.239484,-22.0,125.300,66.760088,0.532802,2.4,1.341641,0.559017,244.50,1.904,1.675539,0.880010,13.0,82,50,2.99,4.89,147.508,161.144,0.272792,0.000000,1.414214,2.87,0.574,2.351916,0.0,0.0,0.0,-1.87,-0.374,-1.577540,0.0,0.0,0.0
12061,2024-01-06,Eua Ncaa,22:00,Arizona,Utah Utes,1.14,5.60,164.5,1.80,1.91,-11.5,1.83,1.83,4nx8m6Pe,0.877193,0.178571,0.555556,0.523560,0.055764,111.792,14.592833,0.130536,1.8,1.643168,0.912871,114.46,1.212,0.123369,0.101790,60.0,98.510,16.194150,0.164391,2.4,1.341641,0.559017,95.90,1.216,0.166973,0.137313,71.0,97,70,1.18,1.37,108.296,0.000,0.935815,0.041931,0.00000

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V1 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11715,05:30,Taiwan Liga T1,Kaohsiung Aquas,Taoyuan Leopards,1.99,Back Home
11759,16:00,Polônia 1. Liga,Sokol Miedzychod,WKK Wroclaw,1.95,Back Home


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
11711,03:30,Taiwan P. League+,Formosa Dreamers,New Taipei Kings,1.67,Back Home
11715,05:30,Taiwan Liga T1,Kaohsiung Aquas,Taoyuan Leopards,1.99,Back Home
11716,03:00,Taiwan Liga T1,Taipei Taishin Mars,Tainan TSG GhostHawks,2.31,Back Home
11718,13:30,Suíça Liga Sb Feminina,Pully F,Alte Kanti Aarau F,1.67,Back Home
11722,13:00,Suíça Liga Sb,Lugano,Sdent BBC Nyon,1.66,Back Home
...,...,...,...,...,...,...
12041,20:00,Eua Ncaa,Oklahoma,Iowa State,1.64,Back Home
12056,23:00,Eua Ncaa,Utah State,Colorado State,1.83,Back Home
12059,22:00,Eua Ncaa,Indiana,Ohio State,2.10,Back Home
12060,22:00,Eua Ncaa,Arizona State,Colorado,2.35,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
11718,13:30,Suíça Liga Sb Feminina,Pully F,Alte Kanti Aarau F,1.87,Back Home
11750,12:15,Turquia Tbl,Balıkesir BŞB,OGM Ormanspor,1.83,Back Home
11911,16:00,Eua Ncaa,Arkansas,Auburn,1.83,Back Home
11915,14:30,Eua Ncaa,Florida,Kentucky,1.83,Back Home
11957,14:30,Áustria Liga Zweite,Kufstein,Worthersee Piraten,1.85,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11715,05:30,Taiwan Liga T1,Kaohsiung Aquas,Taoyuan Leopards,1.99,Back Home
11731,15:00,Eua Ncaa,Wagner,St. Francis (PA),1.63,Back Home
11763,13:00,Portugal Lpb,Ovarense,Vitória SC,1.58,Back Home
11773,13:30,Polônia Liga De Basquete,Spojnia Stargard,Czarni Slupsk,1.62,Back Home
11805,21:00,Eua Ncaa,Eastern Kentucky,Austin Peay,1.55,Back Home
11832,22:30,Eua Ncaa,Southern Indiana,Tennessee Tech,1.69,Back Home
11845,17:00,Eua Ncaa,Louisiana Monroe,Georgia Southern,1.61,Back Home
11889,15:00,Eslovênia Liga Nova Kbm,Hopsi Polzela,Ilirija,2.80,Back Home
11936,16:30,Itália Liga A,Treviso,Varese,1.61,Back Home
11948,17:00,Brasil Nbb,São Paulo,Bauru,1.57,Back Home
